# LangGraph MCP (Model Context Protocol) 튜토리얼

이 튜토리얼에서는 LangGraph와 MCP(Model Context Protocol)를 통합하여 강력한 AI 에이전트를 구축하는 방법을 배웁니다.

## 학습 목표
- MCP의 개념과 아키텍처 이해
- MultiServerMCPClient를 사용한 다중 서버 관리
- React Agent 및 ToolNode와 MCP 통합
- 실전 예제를 통한 복잡한 에이전트 구축

## 목차
1. **MCP 개요 및 설치**
2. **기본 MCP 서버 생성**
3. **MultiServerMCPClient 설정**
4. **React Agent와 MCP 통합**
5. **ToolNode와 MCP 통합**
6. **다중 MCP 서버 관리**
7. **실전 예제 - 복잡한 에이전트 구축**

## Part 1: MCP 기본 개념

### MCP(Model Context Protocol)란?

MCP는 애플리케이션이 언어 모델에 도구와 컨텍스트를 제공하는 방법을 표준화한 오픈 프로토콜입니다.

#### 주요 특징:
- 🔧 **표준화된 도구 인터페이스**: 일관된 방식으로 도구 정의 및 사용
- 🌐 **다양한 전송 메커니즘**: stdio, HTTP, WebSocket 등 지원
- 🔄 **동적 도구 검색**: 런타임에 도구 자동 검색 및 로드
- 🏗️ **확장 가능한 아키텍처**: 여러 서버를 동시에 연결 가능

### 설치

MCP를 사용하기 위해 필요한 패키지를 설치합니다:

In [2]:
import nest_asyncio
from typing import List, Dict, Any

from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import InMemorySaver

# MCP
from langchain_mcp_adapters.client import MultiServerMCPClient

In [3]:
# 환경 변수 설정
from dotenv import load_dotenv

load_dotenv(override=True)

# 비동기 호출 활성화
nest_asyncio.apply()

In [4]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("LangGraph-MCP")

LangSmith 추적을 시작합니다.
[프로젝트명]
LangGraph-MCP


## Part 2: 기본 MCP 서버 생성

MCP 서버는 도구를 제공하는 독립적인 프로세스입니다. FastMCP를 사용하여 간단한 서버를 만들어봅시다.

- `server/mcp_server_local.py`
- `server/mcp_server_rag.py`
- `server/mcp_server_remote.py`

## Part 3: MultiServerMCPClient 설정

MultiServerMCPClient를 사용하면 여러 MCP 서버를 동시에 관리할 수 있습니다.

In [5]:
# MultiServerMCPClient 설정 예제
async def setup_mcp_client(server_configs: List[Dict[str, Any]]):
    """MCP 클라이언트를 설정하고 도구를 가져옵니다."""

    # MCP 클라이언트 생성
    client = MultiServerMCPClient(server_configs)

    # 도구 가져오기
    tools = await client.get_tools()

    print(f"✅ {len(tools)} 개의 MCP 도구가 로드되었습니다:")
    for tool in tools:
        print(f"  - {tool.name}")

    return client, tools

In [ ]:
# 서버 구성 정의
server_configs = {
    "weather": {
        "command": "uv",
        "args": ["run", "python", "server/mcp_server_local.py"],
        "transport": "stdio",
    },
}

# MCP 클라이언트 생성
client, tools = await setup_mcp_client(server_configs=server_configs)

In [ ]:
# LLM 설정
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# React Agent 생성
agent = create_react_agent(
    llm, tools, checkpointer=InMemorySaver()  # 상태 저장을 위한 체크포인터
)

In [ ]:
from langchain_teddynote.messages import astream_graph, random_uuid
from langchain_core.runnables import RunnableConfig

config = RunnableConfig(configurable={"thread_id": random_uuid()})

response = await astream_graph(
    agent,
    inputs={"messages": [("human", "안녕하세요. 서울의 날씨를 알려주세요.")]},
    config=config,
)

### HTTP 전송 방식 사용

원격 서버나 HTTP 엔드포인트를 사용하는 경우 먼저 Remote MCP 서버를 구동해야 합니다.

```bash
uv run python server/mcp_server_remote.py
```

In [ ]:
# HTTP 기반 MCP 서버 설정 예제
http_server_config = {
    "current_time": {
        "url": "http://127.0.0.1:8002/mcp",
        "transport": "streamable_http",
    },
}

# MCP 클라이언트 생성
client, http_tools = await setup_mcp_client(server_configs=http_server_config)

In [ ]:
# LLM 설정
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# React Agent 생성
agent = create_react_agent(
    llm, http_tools, checkpointer=InMemorySaver()  # 상태 저장을 위한 체크포인터
)

In [ ]:
from langchain_teddynote.messages import astream_graph, random_uuid
from langchain_core.runnables import RunnableConfig

config = RunnableConfig(configurable={"thread_id": random_uuid()})

response = await astream_graph(
    agent,
    inputs={"messages": [("human", "안녕하세요. 현재 시간을 알려주세요.")]},
    config=config,
)

### MCP Inspector 사용

`npx @modelcontextprotocol/inspector` 명령어를 실행하여 브라우저에서 MCP 서버를 테스트할 수 있습니다.

![mcp_inspector](./assets/mcp-inspector.png)

In [11]:
# HTTP 기반 MCP 서버 설정 예제
http_server_config = {
    "rag": {
        "command": "uv",
        "args": ["run", "python", "server/mcp_server_rag.py"],
        "transport": "stdio",
    },
}

# MCP 클라이언트 생성
client, rag_tools = await setup_mcp_client(server_configs=http_server_config)

✅ 1 개의 MCP 도구가 로드되었습니다:
  - retrieve


In [12]:
# LLM 설정
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# React Agent 생성
rag_agent = create_react_agent(
    llm, rag_tools, checkpointer=InMemorySaver()  # 상태 저장을 위한 체크포인터
)

In [13]:
from langchain_teddynote.messages import astream_graph, random_uuid
from langchain_core.runnables import RunnableConfig

config = RunnableConfig(configurable={"thread_id": random_uuid()})


_ = await astream_graph(
    rag_agent,
    inputs={
        "messages": [
            (
                "human",
                "삼성전자가 개발한 생성형 AI 의 이름은? mcp 서버를 사용해서 검색해주세요.",
            )
        ]
    },
    config=config,
)


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 

🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
SPRi AI Brief |
2023-12월호
삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개
KEY Contents
n 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성
AI 모델 ‘삼성 가우스’를 공개
n 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유
£언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원
n 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델
‘삼성 가우스’를 최초 공개
∙ 정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에
최적화된 크기의 모델 선택이 가능
∙ 삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며,
온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유
∙ 삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며, 생성 AI 모델을 다양한 제품에
단계적으로 탑재할 계획
n 삼성 가우스는 △텍스트를 생성하는 언어모델 △코드를 생성하는 코드 모델 △이미지를 생성하는
이미지 모델의 3개 모델로 구성
∙ 언어 모델은 클라우드와 온디바이스 대상 다양한 모델로 구성되며, 메일 작성, 문서 요약, 번역 업무의
처리를 지원
∙ 코드 모델 기반의 AI 코딩 어시스턴트 ‘코드아이(code.i)’는 대화형 인터페이스로 서비스를 제공하며
사내 소프트웨어 개발에 최적화
∙ 이미지 모델은 창의적인 이미지를 생성하고 기존 이미지를 원하는 대로 바꿀 수 

In [15]:
_ = await astream_graph(
    rag_agent,
    inputs={
        "messages": [
            (
                "human",
                "구글이 Anthropic 에 투자하기로 한 금액을 검색해줘",
            )
        ]
    },
    config=config,
)


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 

🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
∙ 구글은 앤스로픽 외에도 AI 동영상 제작 도구를 개발하는 런웨이(Runway)와 오픈소스 소프트웨어
기업 허깅 페이스(Hugging Face)에도 투자
∙ 구글은 챗GPT의 기반 기술과 직접 경쟁할 수 있는 차세대 LLM ‘제미니(Gemini)’를 포함한 자체 AI
시스템 개발에도 수십억 달러를 투자했으며, 2024년 제미니를 출시할 계획
☞ 출처 : The Wall Street Journal, Google Commits $2 Billion in Funding to AI Startup Anthropic, 2023.10.27.
Bloomberg, AI Startup Anthropic to Use Google Chips in Expanded Partnership, 2023.11.09.
1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육
구글, 앤스로픽에 20억 달러 투자로 생성 AI 협력 강화
KEY Contents
n 구글이 앤스로픽에 최대 20억 달러 투자에 합의하고 5억 달러를 우선 투자했으며, 앤스로픽은
구글과 클라우드 서비스 사용 계약도 체결
n 3대 클라우드 사업자인 구글, 마이크로소프트, 아마존은 차세대 AI 모델의 대표 기업인
앤스로픽 및 오픈AI와 협력을 확대하는 추세
£구글, 앤스로픽에 최대 20억 달러 투자 합의 및 클라우드 서비스 제공
n 구글이 2023년 10월 27일 앤스로픽에 최대 20억 달러를 투자하기로 합의했으며, 이 중 5억
달러를 우선 투자하고 향후 15억 달러를 추가로 투자할 방침
∙ 구글은 2023년 2월 앤스로픽에 이미 5억 5,000만 달러를 투자한 바 있으며, 아마존도 지난 9월
앤스로픽에 최대 40억 달러의 투자 계획을 공개
∙ 한편, 2023년 11월 8일 블룸버

## Part 4: React Agent와 MCP 통합

React Agent는 추론(Reason)과 행동(Act)을 반복하는 패턴을 구현합니다. MCP 도구와 함께 사용하면 강력한 에이전트를 만들 수 있습니다.

In [ ]:
async def create_mcp_react_agent(server_configs: dict):
    """MCP 도구를 사용하는 React Agent를 생성합니다."""

    # MCP 클라이언트 생성 및 도구 가져오기
    client, tools = await setup_mcp_client(server_configs=server_configs)

    # LLM 설정
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

    # React Agent 생성
    agent = create_react_agent(
        llm, tools, checkpointer=InMemorySaver()  # 상태 저장을 위한 체크포인터
    )

    return agent

In [ ]:
# MCP 서버 Config 설정
server_configs = {
    "weather": {
        "command": "uv",
        "args": ["run", "python", "server/mcp_server_local.py"],
        "transport": "stdio",
    },
    "current_time": {
        "url": "http://127.0.0.1:8002/mcp",
        "transport": "streamable_http",
    },
}

# MCP ReAct Agent 생성
agent = await create_mcp_react_agent(server_configs)

In [ ]:
from langchain_core.runnables import RunnableConfig
from langchain_teddynote.messages import random_uuid, astream_graph


config = RunnableConfig(configurable={"thread_id": random_uuid()})

await astream_graph(
    agent, inputs={"messages": [("human", "현재 시간을 알려주세요")]}, config=config
)

await astream_graph(
    agent,
    inputs={"messages": [("human", "현재 서울의 날씨도 알려주세요")]},
    config=config,
)

## Part 5: ToolNode와 MCP 통합

ToolNode를 사용하면 더 세밀한 제어가 가능한 커스텀 워크플로우를 만들 수 있습니다.

In [ ]:
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_teddynote.graphs import visualize_graph
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage
from typing import Annotated, List, Dict, Any
from langchain_openai import ChatOpenAI
from typing import TypedDict
from langchain_tavily import TavilySearch


# 상태 정의
class AgentState(TypedDict):
    """에이전트 상태"""

    messages: Annotated[List[BaseMessage], add_messages]
    context: Dict[str, Any] = None  # 추가 컨텍스트 정보


async def create_mcp_workflow(server_configs: dict):
    """MCP 도구를 사용하는 커스텀 워크플로우 생성"""

    # MCP 클라이언트 생성
    client, tools = await setup_mcp_client(server_configs=server_configs)

    # Tavily 검색 도구 추가
    tool = TavilySearch(max_results=2)
    tools.append(tool)

    # LLM 설정 (도구 바인딩)
    llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
    llm_with_tools = llm.bind_tools(tools)

    # 워크플로우 그래프 생성
    workflow = StateGraph(AgentState)

    # 노드 정의
    async def agent_node(state: AgentState):
        """LLM을 호출하여 응답 생성"""
        response = await llm_with_tools.ainvoke(state["messages"])
        return {"messages": [response]}

    # ToolNode 생성
    tool_node = ToolNode(tools)

    # 그래프 구성
    workflow.add_node("agent", agent_node)
    workflow.add_node("tools", tool_node)

    workflow.add_edge(START, "agent")
    workflow.add_conditional_edges("agent", tools_condition)
    workflow.add_edge("tools", "agent")
    workflow.add_edge("agent", END)

    # 컴파일
    app = workflow.compile(checkpointer=InMemorySaver())

    visualize_graph(app)

    return app

In [ ]:
# MCP 서버 Config 설정
server_configs = {
    "weather": {
        "command": "uv",
        "args": ["run", "python", "server/mcp_server_local.py"],
        "transport": "stdio",
    },
    "current_time": {
        "url": "http://127.0.0.1:8002/mcp",
        "transport": "streamable_http",
    },
}

In [ ]:
mcp_app = await create_mcp_workflow(server_configs)

In [ ]:
_ = await astream_graph(
    mcp_app, inputs={"messages": [("human", "현재 시간을 알려주세요")]}, config=config
)

In [ ]:
_ = await astream_graph(
    mcp_app,
    inputs={
        "messages": [
            ("human", "오늘 뉴스를 검색해주세요. 검색시 시간을 조회한 뒤 처리하세요.")
        ]
    },
    config=config,
)

## Part 6: 외부 MCP 서버에서 3rd Party 도구 사용하기

**Smithery AI란?**

- 사이트: https://smithery.ai/

- Smithery AI는 AI 에이전트 서비스의 허브 역할을 하는 플랫폼입니다. 

에이전트형 AI(예: 대형 언어 모델)가 외부 도구나 정보와 효율적으로 연결될 수 있도록 설계된 MCP 서버들을 검색하고 배포하는 역할을 수행합니다. 즉, AI가 다양한 외부 서비스와 손쉽게 통신할 수 있게 하는 중개자이자 생태계 허브입니다.

이 모든 서비스의 연결은 **MCP(Model Context Protocol)** 표준 프로토콜을 따릅니다. 이를 통해 AI가 검색, 프로그래밍, 파일 관리, 다양한 API 연동 등 여러 외부 기능을 하나의 허브(플랫폼)만으로 편리하게 활용할 수 있습니다

In [ ]:
# MCP 서버 Config 설정
server_configs = {
    "weather": {
        "command": "uv",
        "args": ["run", "python", "server/mcp_server_local.py"],
        "transport": "stdio",
    },
    "current_time": {
        "url": "http://127.0.0.1:8002/mcp",
        "transport": "streamable_http",
    },
    "context7-mcp": {
        "command": "npx",
        "args": [
            "-y",
            "@smithery/cli@latest",
            "run",
            "@upstash/context7-mcp",
            "--key",
            "7c5b4b8f-cb2a-4c2b-b0e0-d3cc49fc7a85",
        ],
        "transport": "stdio",
    },
}

mcp_app = await create_mcp_workflow(server_configs)

In [ ]:
await astream_graph(
    mcp_app,
    inputs={
        "messages": [
            (
                "human",
                "최신 LangGraph 도큐먼트에서 ReAct Agent 관련 내용을 검색하세요. 그런 다음 Tavily 검색을 수행하는 ReAct Agent를 생성하세요. 사용 LLM=gpt-4.1-mini",
            )
        ]
    },
    config=config,
)